In [1]:
import math
import numpy as np

MAX_DEPTH = 500
MIN_SAMPLE = 1

TRAINING_SIZE = 10000
TEST_SIZE = 1000

D = 10

def var(cnt, s1, s2):
    assert cnt != 0
    return s2 / cnt - (s1 / cnt) ** 2

def cov(x, y):
    return np.mean(x * y) - np.mean(x) * np.mean(y)

def corr(x, y):
    if np.var(y) == 0:
        return 0
    if np.var(x) == 0:
        return 0
    return cov(x, y) / math.sqrt(np.var(x) * np.var(y))

def generate_data(size, d):
    assert size > 0 and d > 5
    x = []
    x.append(np.random.normal(3, 1, size))
    x.append(np.random.normal(-2, 1, size))
    x.append(x[0] + 2 * x[1])
    x.append((x[1] + 2)**2)
    x.append(np.random.binomial(n=1, p=0.8, size=size))
    for _ in range(d - 5):
        x.append(np.random.normal(0, 0.1, size))
    return x

def transpose(temp):
    temp = np.array(temp)
    return temp.T

def compute_y(x):
    y = 4 - 3 * x[0] * x[0] + x[2] - 0.01 * x[3] + x[1] * x[4] + np.random.normal(0, 0.1, len(x[0]))
    return y

class DecisionTree():    

    superfluous = {}  

    def __init__(self, x, y, d):
        self.x = x
        self.y = y
        
        self.depth = d 
        self.child = len(self.x[0]) <= MIN_SAMPLE or self.depth == MAX_DEPTH
        self.ind = -1
        self.threshold = 0
        self.marker = 0
        self.result = np.mean(y)
        self.left, self.right = None, None

        if not self.child: 
            self.split()
    
    def find_best_feature(self):
        all_corr = [abs(corr(xi, self.y)) for xi in self.x]
        return np.argmax(all_corr)
    
    def find_threshold_split(self, ind):
#         indices = self.x[ind].argsort()
        
#         for i in range(len(self.x)):
#             self.x[i] = self.x[i][indices]
#         self.y = self.y[indices]
        
        threshold = -1
        mx = float('inf')
        
        left = 0
        left_sum = 0
        left_sq_sum = 0

        right = len(self.y)
        right_sum = np.sum(self.y)
        right_sq_sum = (self.y ** 2).sum()
        
        tot = len(self.y)
        
        for i in range(len(self.y) - 1):
            left += 1
            right -= 1
    
            left_sum += self.y[i]
            right_sum -= self.y[i]

            left_sq_sum += self.y[i]**2
            right_sq_sum -= self.y[i]**2
            
            err_left = left / tot * var(left, left_sum, left_sq_sum)
            err_right = right / tot * var(right, right_sum, right_sq_sum)
            
            err = err_left + err_right
            if err < mx:
                threshold, mx = left, err
    
        return threshold
     
    def split(self):
        self.ind = self.find_best_feature()
        self.threshold = self.find_threshold_split(self.ind)
        
        self.marker = self.x[self.ind][self.threshold]
        
        x_left = []
        x_right = []
        for i in range(len(self.x)):
            x_left.append(self.x[i][0:self.threshold])
            x_right.append(self.x[i][self.threshold:len(self.x[i])])
        
        y_left = self.y[0:self.threshold]
        y_right = self.y[self.threshold:len(self.y)]

        self.left = DecisionTree(x_left, y_left, self.depth + 1)
        self.right = DecisionTree(x_right, y_right, self.depth + 1)

    @staticmethod
    def predict(node, arr):
        if node.child:
            return node.result
        if arr[node.ind] < node.marker:
            return DecisionTree.predict(node.left, arr)
        else:
            return DecisionTree.predict(node.right, arr)

    #following method assumes that MAX_DEPTH is set to >= 100
    #which is a comfortable upper bound for the depth
    @staticmethod
    def find_max_depth(node):
        if not node:
            return 0
        return 1 + max(DecisionTree.find_max_depth(node.left), DecisionTree.find_max_depth(node.right))

    
#implemented of mean squared error
def compute_mse(x, y, dt):
    err = 0
    for i in range(len(y)):
        yp = DecisionTree.predict(dt, x[i])
        err += (abs(yp - y[i]) ** 2)
    err = err / len(y)
    return err



x = generate_data(TRAINING_SIZE, D)
test_x = generate_data(TEST_SIZE, D)

xt = transpose(x)
tx = transpose(test_x)

y = compute_y(x)
test_y = compute_y(test_x)

dt = DecisionTree(x, y, 1)
#DecisionTree.get_all_ss(dt)
training_err = compute_mse(xt, y, dt)
#test_err = compute_mse(tx, test_y, dt)


#print('Training Error: ', training_err)
#print('Test Error: ', test_err)
#print('Superfluous:', len(dt.superfluous))

In [2]:
xt[10]

array([ 3.7044277 , -1.51671718,  0.67099334,  0.23356229,  1.        ,
       -0.00975547,  0.14070395, -0.08113514, -0.12962663,  0.03463059])

In [4]:
print(test_y[10])

-25.80143604412483


In [40]:
def dfs(node, marker):
    if not node:
        return
    if node.result == marker:
        print('Yes')
    dfs(node.left, marker)
    dfs(node.right, marker)

In [21]:
dfs(dt, y[310])

0
0
0
1
0
1
3
4
0
0
-1
4
0
-1
3
9
6
2
-1
-1
-1
-1
-1
8
2
-1
-1
6
1
-1
-1
-1
0
1
-1
1
0
-1
-1
2
-1
-1
8
5
9
2
-1
-1
-1
-1
5
0
-1
-1
2
-1
-1
0
0
1
7
5
5
-1
1
-1
-1
-1
2
-1
-1
0
3
-1
5
1
-1
-1
-1
1
-1
-1
0
5
2
-1
-1
-1
1
2
2
-1
-1
-1
5
7
7
-1
0
-1
-1
-1
1
-1
-1
0
0
1
1
7
7
-1
2
-1
-1
5
0
-1
-1
-1
8
-1
0
-1
-1
0
9
8
-1
-1
-1
-1
1
3
6
7
5
2
-1
-1
5
5
-1
-1
-1
-1
7
-1
1
-1
-1
-1
0
2
-1
-1
8
3
5
-1
0
-1
-1
-1
5
2
-1
-1
-1
3
0
7
-1
2
-1
-1
2
-1
-1
6
-1
1
-1
-1
0
5
2
-1
-1
1
5
0
1
-1
-1
0
8
7
5
-1
1
-1
-1
5
-1
-1
-1
1
-1
2
-1
-1
0
-1
-1
1
-1
-1
4
1
1
-1
-1
3
-1
-1
3
0
1
6
6
5
-1
-1
1
-1
-1
-1
0
-1
9
3
-1
-1
-1
1
6
-1
9
-1
0
-1
-1
5
7
-1
-1
5
2
-1
-1
-1
1
-1
-1
0
4
0
1
-1
9
6
2
0
-1
1
-1
-1
-1
-1
-1
8
8
-1
6
-1
-1
3
7
8
0
0
-1
-1
-1
1
-1
-1
-1
0
-1
-1
1
0
0
3
0
7
-1
-1
5
-1
9
2
-1
1
-1
-1
6
-1
1
-1
-1
-1
3
5
0
-1
-1
0
-1
-1
0
-1
0
-1
-1
0
3
-1
2
-1
5
-1
-1
3
5
7
-1
0
0
-1
-1
-1
7
-1
9
-1
2
-1
-1
0
0
-1
1
-1
-1
2
2
2
-1
-1
-1
-1
0
2
9
2
-1
-1
3
1
-1
-1
5
1
-1
-1
-1
3
7
3
-1
0
-1
-1
6
-1
6
-1
-1
3

-1
-1
6
7
1
-1
-1
0
-1
1
-1
-1
1
-1
-1
0
9
5
7
-1
8
-1
1
-1
3
-1
-1
-1
-1
1
9
-1
-1
6
5
5
-1
-1
-1
7
-1
5
-1
6
0
-1
-1
-1
0
1
0
0
-1
-1
7
8
1
-1
-1
-1
-1
0
6
0
0
-1
-1
0
-1
-1
6
0
-1
-1
1
-1
-1
1
1
1
-1
-1
5
-1
0
-1
-1
3
8
-1
1
1
-1
-1
-1
-1
3
0
3
-1
-1
7
-1
7
8
0
-1
-1
-1
-1
0
1
3
-1
-1
0
-1
-1
9
-1
0
-1
-1
1
0
0
1
4
7
-1
5
-1
3
-1
-1
3
0
1
-1
-1
9
3
-1
-1
-1
3
9
7
-1
1
-1
-1
-1
2
2
-1
-1
0
-1
-1
4
7
-1
7
5
8
0
-1
-1
0
-1
-1
-1
-1
3
0
7
0
0
-1
-1
6
-1
3
-1
-1
3
-1
-1
4
4
-1
6
7
2
-1
-1
-1
-1
-1
0
9
1
2
-1
-1
-1
-1
2
-1
-1
1
4
7
-1
6
8
3
-1
-1
-1
-1
1
6
-1
1
-1
-1
9
8
-1
0
-1
9
-1
-1
5
0
-1
-1
7
0
-1
-1
6
-1
3
-1
0
-1
-1
4
0
0
-1
-1
-1
3
1
6
-1
3
-1
-1
0
0
-1
6
0
-1
-1
-1
2
-1
3
-1
0
-1
-1
-1
0
1
4
-1
0
5
6
8
-1
7
-1
3
-1
-1
-1
8
9
-1
1
-1
-1
-1
1
-1
9
-1
-1
0
0
-1
0
9
9
-1
-1
8
1
-1
-1
7
-1
5
-1
-1
3
3
-1
-1
1
-1
-1
0
2
-1
-1
2
8
0
-1
-1
-1
-1
1
4
6
3
-1
-1
7
0
-1
-1
-1
1
0
1
1
-1
-1
9
-1
0
-1
-1
-1
0
0
0
-1
-1
1
3
-1
-1
0
0
-1
-1
9
-1
9
-1
-1
1
2
-1
-1
0
-1
-1
4
3
0
4
9
-1
0
2
-1
-1


9
0
-1
-1
-1
0
2
-1
-1
6
-1
-1
0
0
1
8
0
-1
-1
9
5
1
-1
-1
-1
-1
7
0
-1
-1
8
-1
0
-1
-1
8
7
0
-1
-1
-1
-1
9
0
-1
-1
8
1
0
-1
-1
-1
9
0
-1
-1
2
-1
-1
4
6
0
-1
-1
5
-1
0
-1
-1
1
0
5
5
-1
-1
0
6
3
-1
-1
3
-1
-1
-1
1
8
-1
0
-1
-1
1
2
-1
-1
6
5
-1
-1
-1
5
0
0
8
-1
-1
0
-1
-1
0
-1
-1
1
1
-1
-1
0
0
2
-1
-1
-1
-1
1
4
0
4
6
-1
9
4
7
1
-1
1
-1
-1
0
-1
-1
5
-1
4
6
9
-1
-1
-1
-1
7
7
-1
7
9
-1
-1
-1
-1
0
7
9
-1
8
-1
0
-1
-1
8
0
-1
-1
-1
3
0
-1
3
-1
3
2
-1
-1
-1
6
7
-1
7
-1
-1
-1
4
1
0
-1
-1
8
2
-1
-1
9
-1
7
9
0
-1
-1
-1
-1
0
2
-1
0
-1
-1
8
-1
-1
0
1
0
0
6
6
-1
0
-1
-1
0
-1
-1
0
-1
-1
6
-1
7
-1
0
-1
-1
1
0
7
7
-1
9
-1
2
-1
-1
0
-1
-1
3
5
5
0
-1
-1
-1
-1
0
-1
0
-1
-1
8
0
-1
-1
0
-1
-1
1
0
1
2
-1
-1
7
-1
9
-1
0
-1
-1
3
8
7
-1
2
-1
-1
5
-1
3
-1
-1
-1
3
1
0
9
1
-1
-1
-1
-1
6
7
-1
0
-1
-1
-1
1
0
-1
-1
-1
0
4
1
0
-1
-1
3
7
1
-1
-1
2
0
-1
-1
-1
0
-1
-1
1
7
7
1
-1
-1
5
-1
5
1
-1
-1
-1
0
-1
-1
8
6
-1
-1
0
-1
0
7
-1
-1
-1
1
0
0
-1
-1
0
6
5
-1
7
-1
1
-1
-1
0
-1
-1
7
9
-1
0
-1
-1
5
-1
1
-1
-1
4
1
-1
-1
0
1
0
0


1
-1
0
0
3
8
-1
8
-1
-1
2
-1
-1
9
-1
3
-1
-1
0
-1
-1
0
1
0
-1
-1
8
7
9
1
-1
9
9
-1
-1
-1
-1
-1
1
0
-1
-1
-1
8
-1
0
-1
-1
4
5
6
-1
6
-1
-1
-1
0
1
0
-1
0
-1
-1
0
-1
1
-1
9
-1
9
1
-1
-1
-1
1
-1
8
0
-1
-1
-1
1
0
0
1
4
-1
0
6
-1
1
-1
-1
0
6
-1
7
5
2
-1
-1
0
-1
-1
2
-1
-1
7
-1
-1
0
7
9
-1
8
-1
8
7
-1
-1
-1
0
-1
-1
4
0
-1
-1
6
9
6
-1
-1
0
-1
-1
7
-1
0
-1
-1
1
0
6
-1
2
-1
-1
-1
3
0
3
4
3
-1
-1
1
-1
3
9
-1
9
0
-1
-1
-1
-1
0
0
-1
-1
0
-1
-1
9
4
2
-1
-1
-1
0
-1
-1
0
9
-1
1
-1
-1
4
-1
1
-1
-1
1
-1
3
0
6
8
0
-1
8
-1
-1
-1
-1
1
9
-1
6
1
-1
-1
-1
5
0
-1
-1
0
-1
-1
6
6
0
-1
-1
-1
0
-1
-1
0
3
0
0
8
-1
2
-1
-1
8
-1
-1
6
0
-1
4
-1
5
8
-1
2
-1
-1
-1
9
8
-1
0
8
8
0
-1
-1
-1
0
-1
-1
-1
-1
8
-1
0
-1
-1
3
0
-1
-1
2
-1
-1
0
0
1
0
1
4
0
4
1
3
0
-1
8
-1
5
5
-1
-1
6
-1
-1
1
0
-1
-1
-1
0
2
-1
0
-1
-1
1
-1
0
-1
-1
3
0
0
1
3
7
8
0
-1
-1
-1
1
-1
-1
7
1
-1
-1
-1
-1
3
0
6
0
-1
-1
-1
-1
6
-1
9
0
-1
-1
-1
0
3
7
0
-1
-1
0
-1
-1
-1
0
9
-1
3
-1
9
-1
-1
-1
-1
4
1
7
-1
0
-1
7
-1
-1
5
-1
6
9
9
-1
1
-1
-1
-1
0
-1
-1
0
0
0
-1
0


0
-1
-1
0
9
2
-1
-1
-1
-1
0
8
-1
-1
-1
0
1
-1
6
6
9
-1
-1
-1
5
8
-1
1
-1
-1
-1
0
7
-1
0
1
-1
-1
6
-1
2
-1
-1
-1
0
9
1
-1
-1
-1
3
0
9
5
0
-1
-1
-1
0
-1
7
-1
2
-1
-1
7
8
-1
1
-1
-1
0
-1
-1
-1
1
0
0
1
-1
-1
7
8
6
-1
0
-1
-1
0
-1
-1
0
-1
-1
4
-1
0
0
7
-1
9
-1
9
-1
-1
0
-1
-1
5
7
-1
0
-1
-1
-1
4
0
4
1
3
5
-1
-1
-1
6
-1
0
-1
-1
2
-1
3
4
0
-1
0
-1
-1
9
2
-1
-1
-1
8
1
-1
-1
-1
4
1
0
-1
1
1
-1
-1
-1
4
9
9
0
-1
-1
9
-1
0
-1
-1
8
0
-1
-1
-1
-1
3
0
1
0
0
2
-1
-1
-1
-1
0
2
-1
2
-1
-1
0
-1
-1
3
-1
3
0
-1
6
0
-1
-1
-1
-1
3
-1
-1
0
3
0
5
0
1
7
0
-1
-1
-1
1
-1
1
-1
-1
1
1
-1
-1
3
-1
6
0
-1
3
8
-1
8
-1
-1
-1
9
-1
0
-1
-1
-1
3
9
8
7
6
0
-1
-1
-1
9
-1
-1
-1
8
3
3
-1
-1
-1
-1
-1
1
-1
-1
1
0
-1
-1
8
1
-1
-1
0
-1
-1
0
1
0
4
3
5
5
-1
7
3
-1
-1
2
-1
-1
-1
2
-1
-1
3
0
1
1
-1
-1
5
-1
0
-1
-1
6
3
0
-1
-1
0
-1
-1
6
0
-1
-1
0
-1
-1
0
5
-1
0
-1
-1
5
2
-1
-1
1
-1
-1
3
4
0
2
-1
2
-1
9
0
-1
-1
-1
-1
0
3
1
-1
6
0
-1
-1
-1
-1
1
6
9
-1
0
-1
-1
0
-1
-1
8
5
-1
0
-1
-1
-1
5
-1
8
-1
-1
3
0
0
3
7
-1
0
-1
-1
2
-1
-1
5
0
1
-1
-1

In [22]:
print(len(xt[310]))

10


In [3]:
import math
import numpy as np

MAX_DEPTH = 500
MIN_SAMPLE = 1

TRAINING_SIZE = 10000
TEST_SIZE = 1000

D = 10

def var(cnt, s1, s2):
    assert cnt != 0
    return s2 / cnt - (s1 / cnt) ** 2

def cov(x, y):
    return np.mean(x * y) - np.mean(x) * np.mean(y)

def corr(x, y):
    if np.var(x) == 0:
        return 0
    return cov(x, y) / math.sqrt(np.var(x) * np.var(y))

def generate_data(size, d):
    assert size > 0 and d > 5
    x = []
    x.append(np.random.normal(3, 1, size))
    x.append(np.random.normal(-2, 1, size))
    x.append(x[0] + 2 * x[1])
    x.append((x[1] + 2)**2)
    x.append(np.random.binomial(n=1, p=0.8, size=size))
    for _ in range(d - 5):
        x.append(np.random.normal(0, 0.1, size))
    return x

def transpose(temp):
    temp = np.array(temp)
    return temp.T

def compute_y(x):
    y = 4 - 3 * x[0] * x[0] + x[2] - 0.01 * x[3] + x[1] * x[4] + np.random.normal(0, 0.1, len(x[0]))
    return y

class DecisionTree():    

    superfluous = {}  

    def __init__(self, x, y, d):
        self.x = x
        self.y = y
        
        self.depth = d 
        self.child = len(self.x[0]) <= MIN_SAMPLE or self.depth == MAX_DEPTH or np.var(y) == 0
        self.ind = -1
        self.threshold = 0
        self.marker = 0
        self.result = np.mean(y)
        self.left, self.right = None, None

        if not self.child:
            self.split()
    
    def find_best_feature(self):
        all_corr = [abs(corr(xi, self.y)) for xi in self.x]
        return np.argmax(all_corr)
    
    def find_threshold_split(self, ind):
        indices = self.x[ind].argsort()
        
        for i in range(len(self.x)):
            self.x[i] = self.x[i][indices]
        self.y = self.y[indices]
        
        threshold = -1
        mn = float('inf')

        for i in range(len(Y) - 1):
            left = i
            right = len(Y) - i
            
            fltr_left = x[ind] <= x[ind][i]
            fltr_right = x[ind] > x[ind][i]
            
            var_left = np.var(Y[fltr_left])
            var_right = np.var(Y[fltr_right])
            
            err_left = left / len(Y) * var_left
            err_right = right / len(Y) * var_right
            
            err = err_left + err_right
            
            if err < mn:
                threshold, mn = X[ind][i], err
     
    def split(self):
        self.ind = self.find_best_feature()
        self.threshold = self.find_threshold_split(self.ind)

        fltr_left = self.x[self.ind] <= self.threshold
        fltr_right = self.x[self.ind] > self.threshold
        
        x_left = []
        x_right = []
        
        for i in range(len(self.x)):
            x_left.append(self.x[i][fltr_left])
            x_right.append(self.x[i][fltr_right])
        
        self.left = DecisionTree(x_left, y_left, self.depth + 1)
        self.right = DecisionTree(x_right, y_right, self.depth + 1)

    @staticmethod
    def predict(node, arr):
        if node.child:
            return node.result
        if arr[node.ind] <= node.marker:
            return DecisionTree.predict(node.left, arr)
        else:
            return DecisionTree.predict(node.right, arr)

    #following method assumes that MAX_DEPTH is set to >= 100
    #which is a comfortable upper bound for the depth
    @staticmethod
    def find_max_depth(node):
        if not node:
            return 0
        return 1 + max(DecisionTree.find_max_depth(node.left), DecisionTree.find_max_depth(node.right))

In [236]:
x = generate_data(10, D)
test_x = generate_data(10, D)

xt = transpose(x)
tx = transpose(test_x)

y = compute_y(x)
test_y = compute_y(test_x)

dt = DecisionTree(x, y, 1)
training_err = compute_mse(xt, y, dt)

[ -1.32102963  -3.67362371 -11.42444069  -9.70210972 -17.80186195
 -28.02520936]
[-28.27775723 -36.75668226 -51.85422683 -49.84581107]
[ -1.32102963  -3.67362371 -11.42444069  -9.70210972 -17.80186195]
[-28.02520936]
[ -1.32102963  -3.67362371 -11.42444069]
[ -9.70210972 -17.80186195]
[-11.42444069  -3.67362371]
[-1.32102963]
[ -3.67362371 -11.42444069]
[]
[ -3.67362371 -11.42444069]
[]
[ -3.67362371 -11.42444069]
[]
[ -3.67362371 -11.42444069]
[]
[ -3.67362371 -11.42444069]
[]
[ -3.67362371 -11.42444069]
[]
[ -3.67362371 -11.42444069]
[]
[ -3.67362371 -11.42444069]
[]
[ -3.67362371 -11.42444069]
[]
[ -3.67362371 -11.42444069]
[]
[ -3.67362371 -11.42444069]
[]
[ -3.67362371 -11.42444069]
[]
[ -3.67362371 -11.42444069]
[]
[ -3.67362371 -11.42444069]
[]
[ -3.67362371 -11.42444069]
[]
[ -3.67362371 -11.42444069]
[]
[ -3.67362371 -11.42444069]
[]
[ -3.67362371 -11.42444069]
[]
[ -3.67362371 -11.42444069]
[]
[ -3.67362371 -11.42444069]
[]
[ -3.67362371 -11.42444069]
[]
[ -3.67362371 -11.424

[ -3.67362371 -11.42444069]
[]
[ -3.67362371 -11.42444069]
[]
[ -3.67362371 -11.42444069]
[]
[ -3.67362371 -11.42444069]
[]
[ -3.67362371 -11.42444069]
[]
[ -3.67362371 -11.42444069]
[]
[ -3.67362371 -11.42444069]
[]
[ -3.67362371 -11.42444069]
[]
[ -3.67362371 -11.42444069]
[]
[ -3.67362371 -11.42444069]
[]
[ -3.67362371 -11.42444069]
[]
[ -3.67362371 -11.42444069]
[]
[ -3.67362371 -11.42444069]
[]
[ -3.67362371 -11.42444069]
[]
[ -3.67362371 -11.42444069]
[]
[ -3.67362371 -11.42444069]
[]
[ -3.67362371 -11.42444069]
[]
[ -3.67362371 -11.42444069]
[]
[ -3.67362371 -11.42444069]
[]
[ -3.67362371 -11.42444069]
[]
[ -3.67362371 -11.42444069]
[]
[ -3.67362371 -11.42444069]
[]
[ -3.67362371 -11.42444069]
[]
[ -3.67362371 -11.42444069]
[]
[ -3.67362371 -11.42444069]
[]
[ -3.67362371 -11.42444069]
[]
[ -3.67362371 -11.42444069]
[]
[ -3.67362371 -11.42444069]
[]
[ -3.67362371 -11.42444069]
[]
[ -3.67362371 -11.42444069]
[]
[ -3.67362371 -11.42444069]
[]
[ -3.67362371 -11.42444069]
[]
[ -3.673

[-17.80186195  -9.70210972]
[]
[-17.80186195  -9.70210972]
[]
[-17.80186195  -9.70210972]
[]
[-17.80186195  -9.70210972]
[]
[-17.80186195  -9.70210972]
[]
[-17.80186195  -9.70210972]
[]
[-17.80186195  -9.70210972]
[]
[-17.80186195  -9.70210972]
[]
[-17.80186195  -9.70210972]
[]
[-17.80186195  -9.70210972]
[]
[-17.80186195  -9.70210972]
[]
[-17.80186195  -9.70210972]
[]
[-17.80186195  -9.70210972]
[]
[-17.80186195  -9.70210972]
[]
[-17.80186195  -9.70210972]
[]
[-17.80186195  -9.70210972]
[]
[-17.80186195  -9.70210972]
[]
[-17.80186195  -9.70210972]
[]
[-17.80186195  -9.70210972]
[]
[-17.80186195  -9.70210972]
[]
[-17.80186195  -9.70210972]
[]
[-17.80186195  -9.70210972]
[]
[-17.80186195  -9.70210972]
[]
[-17.80186195  -9.70210972]
[]
[-17.80186195  -9.70210972]
[]
[-17.80186195  -9.70210972]
[]
[-17.80186195  -9.70210972]
[]
[-17.80186195  -9.70210972]
[]
[-17.80186195  -9.70210972]
[]
[-17.80186195  -9.70210972]
[]
[-17.80186195  -9.70210972]
[]
[-17.80186195  -9.70210972]
[]
[-17.801

[-49.84581107 -51.85422683 -36.75668226]
[]
[-49.84581107 -51.85422683 -36.75668226]
[]
[-49.84581107 -51.85422683 -36.75668226]
[]
[-49.84581107 -51.85422683 -36.75668226]
[]
[-49.84581107 -51.85422683 -36.75668226]
[]
[-49.84581107 -51.85422683 -36.75668226]
[]
[-49.84581107 -51.85422683 -36.75668226]
[]
[-49.84581107 -51.85422683 -36.75668226]
[]
[-49.84581107 -51.85422683 -36.75668226]
[]
[-49.84581107 -51.85422683 -36.75668226]
[]
[-49.84581107 -51.85422683 -36.75668226]
[]
[-49.84581107 -51.85422683 -36.75668226]
[]
[-49.84581107 -51.85422683 -36.75668226]
[]
[-49.84581107 -51.85422683 -36.75668226]
[]
[-49.84581107 -51.85422683 -36.75668226]
[]
[-49.84581107 -51.85422683 -36.75668226]
[]
[-49.84581107 -51.85422683 -36.75668226]
[]
[-49.84581107 -51.85422683 -36.75668226]
[]
[-49.84581107 -51.85422683 -36.75668226]
[]
[-49.84581107 -51.85422683 -36.75668226]
[]
[-49.84581107 -51.85422683 -36.75668226]
[]
[-49.84581107 -51.85422683 -36.75668226]
[]
[-49.84581107 -51.85422683 -36.7

[-49.84581107 -51.85422683 -36.75668226]
[]
[-49.84581107 -51.85422683 -36.75668226]
[]
[-49.84581107 -51.85422683 -36.75668226]
[]
[-49.84581107 -51.85422683 -36.75668226]
[]
[-49.84581107 -51.85422683 -36.75668226]
[]
[-49.84581107 -51.85422683 -36.75668226]
[]
[-49.84581107 -51.85422683 -36.75668226]
[]
[-49.84581107 -51.85422683 -36.75668226]
[]
[-49.84581107 -51.85422683 -36.75668226]
[]
[-49.84581107 -51.85422683 -36.75668226]
[]
[-49.84581107 -51.85422683 -36.75668226]
[]
[-49.84581107 -51.85422683 -36.75668226]
[]
[-49.84581107 -51.85422683 -36.75668226]
[]
[-49.84581107 -51.85422683 -36.75668226]
[]
[-49.84581107 -51.85422683 -36.75668226]
[]
[-49.84581107 -51.85422683 -36.75668226]
[]
[-49.84581107 -51.85422683 -36.75668226]
[]
[-49.84581107 -51.85422683 -36.75668226]
[]
[-49.84581107 -51.85422683 -36.75668226]
[]
[-49.84581107 -51.85422683 -36.75668226]
[]
[-49.84581107 -51.85422683 -36.75668226]
[]
[-49.84581107 -51.85422683 -36.75668226]
[]
[-49.84581107 -51.85422683 -36.7

In [230]:
def compute_mse(x, y, dt):
    err = 0
    for i in range(len(y)):
        yp = DecisionTree.predict(dt, x[i])
        print(yp, ' ', y[i], ' ', i)
        err += (abs(yp - y[i]) ** 2)
    err = err / len(y)
    return err

In [231]:
compute_mse(xt, y, dt)

-51.274393463792045   -43.54695280885089   0
-21.95118863671676   -25.481679922868363   1
-21.95118863671676   -10.360269418700431   2
-21.95118863671676   -14.879237752548233   3
-21.95118863671676   -23.107478224243803   4
-21.95118863671676   -18.645593872220537   5
-51.274393463792045   -44.16620804317756   6
-21.95118863671676   -17.01785322325838   7
-51.274393463792045   -38.378895358425666   8
nan   -71.89733222409957   9


nan

In [39]:
dfs(dt, y[37])

0
0
0
1
0
1
0
4
0
8
0
-1
-1
-1
3
1
-1
-1
-1
3
0
3
2
6
-1
6
6
5
-1
8
-1
8
-1
-1
-1
6
-1
-1
7
5
7
-1
1
-1
-1
-1
0
-1
8
3
-1
-1
-1
8
5
-1
6
-1
-1
1
-1
-1
4
-1
1
0
6
5
-1
0
-1
-1
-1
-1
0
-1
8
2
0
-1
-1
-1
-1
8
9
-1
8
-1
-1
-1
4
3
-1
-1
1
4
-1
3
0
7
2
-1
-1
-1
1
-1
-1
-1
4
-1
1
0
3
0
-1
1
-1
-1
6
0
-1
-1
2
-1
-1
5
6
-1
0
-1
-1
-1
0
9
1
-1
-1
-1
0
6
-1
7
8
1
-1
-1
-1
-1
-1
0
4
0
1
5
7
6
8
-1
1
-1
-1
-1
6
-1
-1
-1
0
7
-1
1
-1
-1
-1
1
9
3
8
0
-1
-1
2
-1
-1
2
-1
-1
-1
3
1
-1
-1
1
-1
-1
1
0
3
7
1
2
-1
1
8
-1
1
-1
-1
2
-1
-1
6
6
2
-1
-1
-1
-1
8
-1
0
-1
-1
7
5
7
3
1
8
1
-1
-1
-1
-1
-1
1
-1
-1
5
2
-1
-1
2
-1
-1
5
6
1
-1
-1
7
-1
3
1
-1
-1
8
6
-1
-1
-1
-1
4
-1
1
3
7
5
-1
0
-1
5
5
1
-1
-1
-1
5
-1
2
-1
-1
-1
1
-1
-1
0
2
-1
-1
-1
0
1
3
0
8
-1
0
-1
6
-1
2
-1
-1
6
0
2
-1
-1
5
9
0
-1
-1
0
-1
-1
-1
-1
6
8
-1
5
8
-1
1
-1
-1
-1
1
-1
-1
2
-1
-1
1
3
0
2
-1
-1
9
8
-1
0
3
-1
-1
-1
6
5
-1
0
-1
-1
-1
0
5
-1
0
-1
-1
-1
8
9
7
9
-1
-1
-1
1
-1
-1
-1
4
0
1
1
0
-1
-1
0
-1
8
-1
-1
7
1
5
-1
0
-1
-1
-1
1
-1
-1
1
0
9
-1
-1
0

0
1
0
1
4
0
1
9
-1
1
-1
-1
1
8
9
3
8
1
-1
-1
0
-1
-1
-1
9
-1
-1
1
-1
-1
-1
3
0
0
1
3
1
-1
-1
-1
0
0
-1
-1
6
-1
6
8
-1
2
-1
-1
-1
6
8
8
-1
-1
-1
-1
0
-1
-1
-1
0
1
4
9
-1
6
1
-1
-1
-1
1
9
-1
7
0
-1
-1
-1
0
8
-1
8
-1
-1
8
2
-1
-1
0
-1
-1
0
1
3
6
0
-1
-1
-1
7
2
-1
-1
-1
1
6
8
-1
2
-1
-1
0
-1
-1
3
6
3
-1
-1
0
-1
-1
0
-1
-1
4
0
-1
2
-1
-1
3
0
0
5
0
-1
-1
1
5
-1
-1
8
-1
0
-1
-1
3
8
-1
2
-1
-1
3
8
0
-1
-1
-1
-1
0
9
7
-1
1
-1
-1
5
0
-1
-1
7
7
5
-1
-1
-1
3
-1
-1
1
7
0
-1
-1
1
-1
-1
-1
0
5
9
3
-1
-1
2
-1
-1
5
3
-1
-1
2
-1
-1
5
9
0
-1
-1
0
-1
-1
0
-1
-1
1
4
-1
3
3
3
1
7
-1
5
-1
-1
0
-1
-1
2
-1
-1
0
6
0
-1
-1
-1
-1
-1
3
8
8
-1
3
-1
-1
-1
7
2
-1
-1
-1
4
0
1
4
0
3
1
7
5
0
-1
-1
8
0
-1
-1
0
-1
-1
-1
-1
-1
8
8
-1
2
-1
-1
-1
6
0
9
9
-1
2
-1
-1
2
-1
-1
-1
4
-1
-1
0
0
0
-1
-1
1
-1
-1
0
7
5
-1
0
-1
-1
-1
8
-1
0
-1
-1
0
1
1
0
-1
-1
7
-1
-1
1
0
0
1
-1
-1
3
-1
-1
0
-1
-1
9
0
-1
-1
5
-1
0
-1
-1
1
0
3
0
6
-1
1
-1
-1
0
-1
-1
3
0
-1
0
6
-1
7
-1
0
-1
-1
-1
-1
9
7
-1
7
1
-1
-1
-1
8
-1
0
-1
-1
1
0
0
-1
-1
2
-1
-1
9


1
-1
-1
4
3
0
0
-1
-1
1
-1
-1
0
0
0
-1
-1
-1
-1
1
0
1
0
-1
6
-1
5
-1
-1
6
9
9
-1
5
-1
2
1
-1
-1
-1
-1
0
-1
-1
1
5
-1
2
-1
-1
6
6
0
-1
-1
-1
-1
0
3
9
-1
1
-1
-1
2
-1
-1
5
-1
9
-1
-1
0
3
0
1
4
0
4
-1
0
-1
-1
9
4
1
-1
-1
0
-1
-1
5
0
-1
8
-1
-1
7
-1
7
0
-1
-1
-1
3
0
8
5
-1
-1
-1
0
6
6
-1
-1
-1
6
0
-1
-1
-1
0
-1
9
5
7
-1
-1
-1
-1
0
4
3
1
-1
-1
8
0
-1
-1
-1
1
6
3
0
-1
0
-1
-1
0
-1
-1
-1
6
0
-1
-1
9
0
6
-1
-1
-1
-1
3
9
0
-1
-1
8
0
-1
-1
8
1
-1
3
-1
6
2
0
-1
-1
-1
0
-1
-1
9
-1
8
-1
7
0
-1
-1
-1
0
-1
-1
3
4
9
1
-1
-1
-1
1
1
-1
-1
0
-1
7
6
0
-1
2
-1
-1
-1
0
-1
0
-1
-1
9
1
-1
-1
7
0
-1
-1
0
-1
-1
3
0
1
0
7
2
-1
2
-1
-1
0
-1
-1
1
-1
0
-1
-1
0
8
-1
-1
0
0
-1
-1
3
0
-1
0
-1
-1
-1
1
0
3
1
-1
-1
1
-1
-1
1
6
2
-1
-1
-1
6
2
-1
-1
-1
9
8
-1
3
-1
-1
6
0
-1
-1
-1
3
0
8
0
-1
-1
-1
5
5
0
-1
-1
-1
-1
-1
1
0
3
4
0
7
6
0
-1
-1
5
7
-1
-1
-1
-1
9
1
-1
-1
-1
3
0
8
0
-1
0
-1
2
-1
-1
5
-1
0
-1
-1
1
-1
-1
0
0
-1
-1
3
7
-1
0
-1
-1
8
-1
3
-1
-1
6
8
6
0
-1
-1
0
-1
-1
9
-1
8
2
-1
-1
-1
7
-1
0
-1
-1
3
4
4
-1
4
9
0
-1
-1
-

-1
4
1
-1
-1
1
0
0
-1
-1
5
-1
1
9
-1
1
-1
-1
6
6
-1
-1
-1
7
1
0
-1
8
5
0
-1
-1
-1
-1
8
-1
0
-1
-1
7
5
1
-1
-1
-1
-1
0
4
7
7
1
-1
-1
-1
-1
3
0
8
2
-1
0
-1
-1
8
8
-1
-1
-1
9
8
-1
9
3
0
-1
-1
-1
1
1
-1
-1
-1
7
8
1
-1
-1
-1
-1
0
2
-1
-1
9
1
-1
-1
0
-1
-1
4
0
7
-1
5
-1
-1
3
3
9
-1
-1
6
-1
-1
7
-1
5
-1
1
-1
-1
0
1
3
6
7
-1
5
-1
5
-1
-1
-1
0
0
-1
-1
-1
1
-1
-1
1
5
-1
3
5
6
1
-1
-1
-1
-1
-1
7
-1
-1
4
0
1
-1
1
1
-1
-1
-1
3
0
3
0
-1
-1
-1
5
0
-1
-1
0
3
-1
0
-1
-1
5
7
-1
-1
-1
-1
1
0
8
9
3
6
-1
0
-1
-1
-1
-1
9
-1
0
-1
-1
3
3
0
8
2
-1
5
-1
-1
-1
8
6
-1
6
0
-1
-1
0
0
-1
-1
-1
-1
5
9
5
-1
3
-1
-1
1
-1
-1
-1
-1
0
3
0
1
-1
-1
1
0
-1
0
-1
-1
0
-1
-1
3
3
0
-1
-1
-1
5
3
-1
-1
-1
0
1
6
0
2
-1
-1
8
7
0
-1
-1
0
-1
-1
-1
-1
0
0
0
-1
-1
9
6
9
-1
3
0
-1
-1
-1
0
-1
-1
0
-1
-1
6
0
-1
-1
-1
6
0
-1
-1
-1
1
0
0
4
6
5
1
-1
1
-1
-1
-1
9
-1
3
6
-1
8
-1
9
-1
1
-1
-1
-1
1
0
3
3
8
-1
9
-1
-1
-1
-1
0
-1
8
3
8
8
-1
-1
-1
1
-1
-1
1
-1
-1
0
1
0
3
-1
-1
6
0
-1
-1
0
-1
-1
5
9
0
-1
-1
-1
0
5
0
-1
-1
-1
2
-1
-1
1
-1
3
7
-1
9
-1


-1
6
-1
-1
1
8
1
-1
-1
0
-1
-1
-1
-1
1
4
-1
8
7
-1
-1
7
7
-1
5
0
-1
-1
-1
0
-1
-1
0
-1
5
0
-1
0
3
-1
-1
-1
8
-1
8
-1
-1
0
3
1
3
7
-1
-1
7
3
6
-1
0
-1
-1
-1
-1
-1
-1
0
0
7
-1
6
6
-1
-1
9
-1
8
-1
9
-1
-1
1
-1
-1
-1


In [144]:
def dfs(node, marker, path, ind, markers):
    if not node:
        return
    if node.result == marker:
        print(path)
        print(ind)
        print(markers)
        return
    dfs(node.left, marker, path + ['Left'], ind + [node.ind], markers + [node.marker])
    dfs(node.right, marker, path + ['Right'], ind + [node.ind], markers + [node.marker])

In [90]:
cnt = 0
def coul(node):
    if not node:
        return
    if node.child:
        global cnt
        cnt += 1
        return
    coul(node.left)
    coul(node.right)
coul(dt)
print(cnt)

10000


In [46]:
xt[37]

array([ 1.88811952, -1.03445861, -0.18079769,  0.93227018,  1.        ,
        0.06596784, -0.09666763, -0.03228091,  0.019024  , -0.04891248])

In [147]:
def predict(node, arr):
    if node.child:
        return node.result
    #print(node.ind)
    if arr[node.ind] < node.marker:
        print('Left ', node.ind, ' ', node.marker)
        return predict(node.left, arr)
    else:
        print('Right ', node.ind, ' ', node.marker)
        return predict(node.right, arr)
print(xt[14])
predict(dt, xt[14])

[ 3.73000361 -2.23007468 -0.73014574  0.05293436  1.         -0.04400858
  0.06643629  0.0681357  -0.17976983  0.00433589]
Right  0   3.471557208110774
Left  0   4.391115345250146
Left  0   3.8758582077751775
Left  1   -2.1195539301126423
Right  0   3.714375666866697
Right  1   -3.1038166246511327
Right  4   1
Left  0   3.8011945865738017
Right  1   -2.970415082376193
Right  1   -2.4906610079000857
Left  0   3.7765284334450477
Left  0   3.7630981351047366


-41.20677569386893

In [148]:
dfs(dt, y[14], [], [], [])

['Right', 'Left', 'Left', 'Left', 'Right', 'Right', 'Left', 'Left', 'Right', 'Left', 'Right', 'Left', 'Right', 'Right']
[0, 0, 0, 1, 0, 1, 4, 0, 4, 0, 1, 0, 3, 2]
[3.471557208110774, 4.391115345250146, 3.8758582077751775, -2.1195539301126423, 3.714375666866697, -3.1038166246511327, 1, 3.8146435359064474, 1, 3.747272888203456, -2.2333095120241726, 3.739670279342769, 0.052934356320019214, -0.7301457402403129]


In [ ]:
DecisionTree.predict(dt, x[i])

In [178]:
np.random.normal(3, 1, 10)

array([1.87265815, 2.54266995, 4.11896802, 0.75041732, 2.23093005,
       3.8757642 , 3.77067403, 4.30423166, 3.75092992, 2.99953248])

In [179]:
np.empty(0)

array([], dtype=float64)

In [192]:
def cold(node):
    if not node:
        return
    print(node.result)
    cold(node.left)
    cold(node.right)
cold(dt)

-27.941270250113604
nan
nan


In [209]:
arr = np.empty(0)

In [210]:
np.append(arr, 5)

array([5.])

In [211]:
arr

array([], dtype=float64)